In [ ]:
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import pandas as pd
from sklearn.preprocessing import LabelEncoder as le
from collections import defaultdict


In [ ]:
df_raw = pd.read_excel("output/keep-sander2017-100-30-1/resampled_df_10_min.xlsx", index_col=[0])

In [ ]:
start_date = pd.to_datetime(f"2017-01-01 00:00:00")
end_date = pd.to_datetime(f"2017-05-01 23:50:00")
training_window_size = 7
horizon_size = 7
model_features = ["day", "weekday", "hour", "window_block"] # Day = day of the month (0-31), hour = hour of the day (0-24), weekday = day in the week (0-7), window_block = window block in the hour (0-5)

baseline_performance = defaultdict(dict)

In [ ]:
df = df_raw[df_raw["timestamp"].between(start_date, end_date)].copy()

label_encoder = le()
df.location = label_encoder.fit_transform(df.location)

In [ ]:
df["day"] = df["timestamp"].dt.day
df["weekday"] = df["timestamp"].dt.dayofweek
df["hour"] = df["timestamp"].dt.hour
df["window_block"] = ((df['timestamp'].dt.minute * 60 + df['timestamp'].dt.second) // 600).astype(int)


In [ ]:
df.head(10)

In [ ]:
train_start_date = start_date
train_end_date = train_start_date + pd.Timedelta(days=training_window_size-1, hours=23, minutes=50)
test_start_date = train_end_date + pd.Timedelta(minutes=10)
test_end_date = test_start_date + pd.Timedelta(days=horizon_size-1, hours=23, minutes=50)

train_mask = df["time"].between(train_start_date, train_end_date)
test_mask = df["time"].between(test_start_date, test_end_date)

# Split the data into train and test sets
X_train = df.loc[train_mask, model_features]
y_train = df.loc[train_mask, "location"]
X_test = df.loc[test_mask, model_features]
y_test = df.loc[test_mask, "location"]

print(f"Training: {train_start_date}-{train_end_date}, testing: {test_start_date}-{test_end_date}.")

In [ ]:
training_data = df.loc[train_mask]
testing_data = df.loc[test_mask]
most_common_locations = training_data.groupby(model_features)['location'].apply(lambda x: x.value_counts().idxmax()).reset_index()

In [ ]:
result_df = testing_data.merge(most_common_locations, how="left", left_on=model_features, right_on=model_features)

features_to_use = model_features[1:]
while result_df['location_y'].isna().sum() > 0:
    print('nan > 0, now trying with features: ', features_to_use)
    most_common_locations = training_data[["location"] + features_to_use].groupby(features_to_use)['location'].apply(lambda x: x.value_counts().idxmax()).reset_index()
    result_df = testing_data.merge(most_common_locations, how="left", left_on=features_to_use, right_on=features_to_use)
    features_to_use = features_to_use[1:]  # Remove the first element to exclude it from the next merge

predictions = result_df.location_y.values.tolist()



In [ ]:
for d in range(horizon_size):
    # Then, evaluate the baseline's predictions and store acc in self.baseline_performance
    this_day_predictions = predictions[d*144:(d+1)*144]
    this_day_actual_values = y_test[d*144:(d+1)*144]
    acc = accuracy_score(this_day_actual_values, this_day_predictions)
    print(f"Acc of baseline: {acc}")

In [ ]:
import DataLoader as DL
from Cluster import Cluster

# Initialize parameters.
data_source = "google_maps"  # Can be either 'google_maps' or 'routined'.
# hours_offset is used to offset the timestamps to account for timezone differences. For google maps, timestamp comes in GMT+0
# which means that we need to offset it by 2 hours to make it GMT+2 (Dutch timezone). Value must be INT!
hours_offset = 2 # Should be 0 for routined and 2 for google_maps. 
# begin_date and end_date are used to filter the data for your analysis.
begin_date = "2022-01-01"
end_date = "2022-12-30"  # End date is INclusive! 
# FRACTION is used to make the DataFrame smaller. Final df = df * fraction. This solves memory issues, but a value of 1 is preferred.
fraction = 1
# For the heatmap visualization we specify a separate begin_date and end_date (must be between begin_date and end_date).
# For readiness purposes, it it suggested to select between 2 and 14 days.
heatmap_begin_date = "2023-01-20"
heatmap_end_date = "2023-05-28"  # End date is INclusive! Choose a date that lies (preferably 2 days) before end_date to avoid errors. 
# For the model performance class we need to specify the number of training days (range) and testing horizon (also in days)
training_window_size = 100
horizon_size = 30
window_step_size = 1
outputs_folder_name = f"remove-{training_window_size}-{horizon_size}-{window_step_size}" # All of the outputs will be placed in output/outputs_folder_name

In [ ]:
df, _ = DL.load_data(
    data_source,
    begin_date,
    end_date,
    fraction,
    hours_offset,
    outputs_folder_name=outputs_folder_name,
    verbose=True,
    perform_eda=True
)

# Step 2. Run clustering
# First, make an instance of the Cluster class and define its settings.
c = Cluster(
    df,  # Input dataset (with latitude, longitude, timestamp columns)
    outputs_folder_name=outputs_folder_name, 
    verbose=True,  # Do we want to see print statements?
    pre_filter=True,  # Apply filters to the data before the clustering (such as removing moving points)
    post_filter=True,  # Apply filters to the data/clusters after the clustering (such as deleting homogeneous clusters)
    filter_moving=True,  # Do we want to delete the data points where the subject was moving?
    centroid_k=10,  # Number of nearest neighbors to consider for density calculation (for cluster centroids)
    min_unique_days=1,  # If post_filter = True, then delete all clusters that have been visited on less than min_unique_days days.
)

# Then we run the clustering and visualisation
df = (
    c.run_clustering(
        min_samples=200,  # The number of samples in a neighborhood for a point to be considered as a core point
        eps=0.01,  # The maximum distance between two samples for one to be considered as in the neighborhood of the other. 0.01 = 10m
        algorithm="dbscan",  # Choose either 'dbscan' or 'hdbscan'. If 'hdbscan', only min_samples is required.
        # min_cluster_size=50,  # Param of HDBSCAN: the minimum size a final cluster can be. The higher this is, the bigger your clusters will be
    )
    .add_locations_to_original_dataframe(
        export_xlsx=False,  # Export the dataframe to excel file? Useful for analyzing.
        name="test",
    )
    .plot_clusters(
        filter_noise=False,  # Remove the -1 labels (i.e., noise) before plotting the clusters
    )
    
    .df  # These functions return 'self' so we can chain them and easily access the df attribute (for input to further modeling/visualization).
)


In [ ]:
c.df

In [ ]:
c.df_centroids.loc[0].latitude


In [ ]:
c.df_centroids.loc[len(c.df_centroids)] = [0, 0, -1, 10, "black", "noise", 0, 0, 0]

In [ ]:
c.df.source.unique()

## Percentage of window blocks that we have seen before, per day

In [1]:
import pandas as pd

df = pd.read_excel("output/martijn-100-30-1/resampled_df_10_min.xlsx", index_col=[0])

df["day"] = df["timestamp"].dt.day
df["weekday"] = df["timestamp"].dt.dayofweek
df["hour"] = df["timestamp"].dt.hour
df["window_block"] = ((df['timestamp'].dt.minute * 60 + df['timestamp'].dt.second) // 600).astype(int)

In [2]:
df

,timestamp,location,day,weekday,hour,window_block
1,2020-05-01 00:10:00,"39, Florijn",1,4,0,1
2,2020-05-01 00:20:00,"39, Florijn",1,4,0,2
3,2020-05-01 00:30:00,"39, Florijn",1,4,0,3
4,2020-05-01 00:40:00,"39, Florijn",1,4,0,4
5,2020-05-01 00:50:00,"39, Florijn",1,4,0,5
...,...,...,...,...,...,...
65947,2021-08-01 23:10:00,"39, Florijn",1,6,23,1
65948,2021-08-01 23:20:00,"39, Florijn",1,6,23,2
65949,2021-08-01 23:30:00,"39, Florijn",1,6,23,3
65950,2021-08-01 23:40:00,"39, Florijn",1,6,23,4


In [3]:
final_results = {} # Dict with keys (dates) and values (scores)
lookup = {} # Dict with the feature & target variable combinations
current_date = df.head(1).timestamp.dt.date.values[0]
this_day_values = []

for index, row in df.iterrows():
    combi = (row['location'], row['weekday'], row['hour'], row['window_block'])
    # print(f"Data: {combi}, curr date: {current_date}, this day values: {this_day_values}")

    if row['timestamp'].date() == current_date:
        if combi in lookup:
            this_day_values.append(1)
        else:
            this_day_values.append(0)

    else:
        # We need to calculate the final scores for last day, since the new day has started. 
        ones = this_day_values.count(1)
        zeros = this_day_values.count(0)

        try:
            ratio = ones/(ones + zeros)
        except ZeroDivisionError:
            ratio = 0
        # print(f"Day {current_date} has ended, ones: {ones}, zeros: {zeros}, ratio: {ratio}")
        final_results[current_date] = ratio

        current_date = row['timestamp'].date()
        this_day_values = []

        if combi in lookup:
            this_day_values.append(1)
        else:
            this_day_values.append(0)

    if not combi in lookup:
        lookup[combi] = 1
    else:
        lookup[combi] += 1

    

In [4]:
x,y = zip(*sorted(final_results.items()))
df_plot = pd.DataFrame({"Time":x, "Percentage of day seen before":y})

In [5]:
df_plot


,Time,Percentage of day seen before
0,2020-05-01,0.000000
1,2020-05-02,0.000000
2,2020-05-03,0.000000
3,2020-05-04,0.000000
4,2020-05-05,0.000000
...,...,...
452,2021-07-27,1.000000
453,2021-07-28,1.000000
454,2021-07-29,1.000000
455,2021-07-30,1.000000


In [6]:
import plotly.express as px
import plotly.graph_objects as go
from scipy import signal

# fig = px.line(df_plot, x="Time", y="Percentage of day seen before", title='How common are the days?')
# fig.show()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_plot.Time.values.tolist(),
    y=signal.savgol_filter(df_plot['Percentage of day seen before'].values.tolist(),
                           20, # window size used for filtering
                           3), # order of fitted polynomial
    name='Savitzky-Golay'
))


In [7]:
fig = px.line(df_plot, x="Time", y="Percentage of day seen before", title='How common are the days?')
fig.show()

## Now based on historical distributions of visited locations

Voor elk blok in de dag wil je weten of die samenvalt met wat die persoon vaak op dat moment doet. Dit geeft een beeld van hoe "routinematig" de dag is, en daarmee hoe routinematig iemand leeft (en dus hoe voorspelbaar iemand is). <br><br>

Elk 10-min block vergelijken we met de distributie van bezochte locaties in het verleden; hoe logisch is de huidige locatie in dit 10-min block gegeven de historische distributie van bezochte locaties op dat "moment"? Een moment is een unieke combi van de temporele features: uur vd dag, dag vd week, etc. Elke dag heeft vervolgens 144 probabilities, daarvan nemen we een aggregatie (gemiddelde). Dit is de score voor de dag. 

In [12]:
df = pd.read_excel("output/martijn-100-30-1/resampled_df_10_min.xlsx", index_col=[0])

df["day"] = df["timestamp"].dt.day
df["weekday"] = df["timestamp"].dt.dayofweek
df["hour"] = df["timestamp"].dt.hour
df["window_block"] = ((df['timestamp'].dt.minute * 60 + df['timestamp'].dt.second) // 600).astype(int)

from collections import defaultdict
res = defaultdict(dict)
final_res = defaultdict(dict)
THRESHOLD = 10
this_day_values = []

for idx, row in df.iterrows():
    m = (row['weekday'], row['hour'], row['window_block'])

    if not 'data' in res[m]:
        res[m]['data'] = {}

    try:
        # Calculate the probability of having this location in this moment
        prob = res[m]['data'][row['location']] / sum(res[m]['data'].values())
        # print(f"For {m}, prob of finding {row['location']} in {res[m]['data']} is: {prob}")
    except KeyError:
        # The location is not in the dict, so we default to 0. This happens when the location hasn't been visited before in this moment. 
        # print(f"For {m}, couldnt find {row['location']} in {res[m]['data']}, so prob is: 0")
        prob = 0
    
    this_day_values.append(prob)

    # Now we update the res dict with data of current window block
    if row['location'] in res[m]['data']:
        res[m]['data'][row['location']] += 1
    else:
        res[m]['data'][row['location']] = 1

    # If we now exceeded THRESHOLD, remove the last entry (like a rolling window approach)
    if sum(res[m]['data'].values()) >= THRESHOLD:
        # Minus one for last entry
        res[m]['data'][res[m]['meta']['last_location']] -= 1
        # print(f"Threshold exceeded, removing 1 from {res[m]['meta']['last_location']}")

    res[m]['meta'] = {'last_location':row['location']}

    # Now we check if this window block is the last one of the day. If so, save score for this day and reset variables for next day.
    if row['hour'] == 23 and row['window_block'] == 5:
        avg = sum(this_day_values)/len(this_day_values)
        # print(f"End of the day, values for this day: {this_day_values}, which is an avg of: {avg}")
        final_res[row['timestamp'].date()] = avg
        this_day_values = []

In [13]:
x,y = zip(*sorted(final_res.items()))
df_plot = pd.DataFrame({"Time":x, "Score":y})

In [14]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.line(df_plot, x="Time", y="Score", title='How common are the days?')
fig.show()

In [15]:
from scipy import signal

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_plot.Time.values.tolist(),
    y=signal.savgol_filter(df_plot['Score'].values.tolist(),
                           20, # window size used for filtering
                           3), # order of fitted polynomial
    name='Savitzky-Golay'
))